In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# here
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import sys

In [3]:
    # here
# Path to sears repository
sys.path.append('sears') # noqa
import paraphrase_scorer
import onmt_model
import numpy as np


In [ ]:
# here
ps = paraphrase_scorer.ParaphraseScorer(gpu_id=0)

GPU ID 0


In [7]:
# here
import spacy
nlp = spacy.load('en')
import replace_rules
tokenizer = replace_rules.Tokenizer(nlp)

In [8]:
# here
import time

In [9]:
# here
import pickle

In [10]:
# here
all_data = pickle.load(open('polarity.pkl', 'rb'))

In [11]:
# here
data = all_data['data']
labels = all_data['labels']
label_names = all_data['label_names']
val = all_data['imdb']
val_labels = all_data['imdb_labels']

In [12]:
# import pickle
# pickle.dump({'data': data, 'labels': labels, 'label_names': label_names, 'imdb': val, 'imdb_labels': val_labels}, open('/tmp/polarity.pkl', 'wb'))

In [13]:
data = tokenizer.clean_for_model(data)

In [14]:
# here
# val, val_labels, _ = load_polarity_imdb()
clean_val = tokenizer.clean_for_model(val)

In [15]:
# here
import fasttext

Using TensorFlow backend.


In [16]:
# here
model = fasttext.FastTextClassifier()
w.fit(data, labels, ngram_range=2, epochs=10, maxlen=100)

Average train sequence length: 36
Epoch 1/10
 - 2s - loss: 0.6878 - acc: 0.5882
Epoch 2/10
 - 2s - loss: 0.6299 - acc: 0.7926
Epoch 3/10
 - 2s - loss: 0.5024 - acc: 0.8827
Epoch 4/10
 - 2s - loss: 0.3702 - acc: 0.9326
Epoch 5/10
 - 2s - loss: 0.2674 - acc: 0.9630
Epoch 6/10
 - 2s - loss: 0.1947 - acc: 0.9807
Epoch 7/10
 - 2s - loss: 0.1438 - acc: 0.9885
Epoch 8/10
 - 2s - loss: 0.1082 - acc: 0.9930
Epoch 9/10
 - 2s - loss: 0.0828 - acc: 0.9958
Epoch 10/10
 - 2s - loss: 0.0645 - acc: 0.9971


In [17]:
# (model.predict(clean_val) == val_labels).mean()

In [18]:
# val_for_onmt = [' '.join([a.text for a in x]) for x in nlp.tokenizer.pipe(val)]
# val_for_onmt = [onmt_model.clean_text(x, only_upper=False) for x in val_for_onmt]

In [19]:
# here
right = np.where(model.predict(clean_val) == val_labels)[0]

In [20]:
# here
right_preds = np.array([val_labels[i] for i in right])

In [21]:
# here
def find_flips(instance, model, topk=10, threshold=-10, ):
    orig_pred = model.predict([instance])[0]
    instance_for_onmt = onmt_model.clean_text(' '.join([x.text for x in nlp.tokenizer(instance)]), only_upper=False)
    paraphrases = ps.generate_paraphrases(instance_for_onmt, topk=topk, edit_distance_cutoff=4, threshold=threshold)
    texts = tokenizer.clean_for_model(tokenizer.clean_for_humans([x[0] for x in paraphrases]))
    preds = model.predict(texts)
    fs = [(texts[i], paraphrases[i][1]) for i in np.where(preds != orig_pred)[0]]
    return fs

In [ ]:
# here
# compute SEA
import collections
orig_scores = {}
flips = collections.defaultdict(lambda: [])
print(len(right))
for i, idx in enumerate(right):
    if i % 100 == 0:
        print(i)
    if val[idx] in flips:
        continue
    fs = find_flips(val[idx], model, topk=100, threshold=-10)
    flips[val[idx]].extend([x[0] for x in fs])


In [23]:
# here
right_val = [clean_val[i] for i in right]

In [24]:
# here
tr2 = replace_rules.TextToReplaceRules(nlp, right_val, [], min_freq=0.005, min_flip=0.00, ngram_size=4)

In [25]:
# here
# extract rules from SEA
frequent_rules = []
rule_idx = {}
rule_flips = {}
for z, f in enumerate(flips):
    rules = tr2.compute_rules(f, flips[f], use_pos=True, use_tags=True)
    for rs in rules:
        for r in rs:
            if r.hash() not in rule_idx:
                i = len(rule_idx)
                rule_idx[r.hash()] = i
                rule_flips[i] = []
                frequent_rules.append(r)
            i = rule_idx[r.hash()]
            rule_flips[i].append(z)
    if z % 500 == 0:
        print (z)

0
500


In [26]:
# here
token_right = tokenizer.tokenize(right_val)

In [27]:
# here
model_preds = {}

In [28]:
# len(frequent_rules)

In [29]:
# here
a = time.time()
rule_flips = {}
rule_other_texts = {}
rule_other_flips = {}
rule_applies = {}
for i, r in enumerate(frequent_rules):
    idxs = list(tr2.get_rule_idxs(r))
    to_apply = [token_right[x] for x in idxs]
    applies, nt = r.apply_to_texts(to_apply, fix_apostrophe=False)
    applies = [idxs[x] for x in applies]
    old_texts = [right_val[i] for i in applies]
    old_labels = right_preds[applies]
    to_compute = [x for x in nt if x not in model_preds]
    if to_compute:
        preds = model.predict(to_compute)
        for x, y in zip(to_compute, preds):
            model_preds[x] = y
    new_labels = np.array([model_preds[x] for x in nt])
    where_flipped = np.where(new_labels != old_labels)[0]
    flips = sorted([applies[x] for x in where_flipped])
    rule_flips[i] = flips
    rule_other_texts[i] = nt
    rule_other_flips[i] = where_flipped
    rule_applies[i] = applies
    if i % 5000 == 0:
        print(i)
print(time.time() - a)

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000
200000
205000
210000
215000
220000
225000
230000
235000
273.9192931652069


In [30]:
# here
really_frequent_rules = [i for i in range(len(rule_flips)) if len(rule_flips[i]) > 1]

In [31]:
# to_compute_score = collections.defaultdict(lambda: set())
# for i in really_frequent_rules:
#     orig_texts =  [right_val[z] for z in rule_applies[i]]
#     new_texts = rule_other_texts[i]
#     for o, n in zip(orig_texts, new_texts):
#         to_compute_score[o].add(n)

In [32]:
# here
threshold = -7.15

In [33]:
# here
orig_scores = {}
for i, t in enumerate(right_val):
    orig_scores[i] = ps.score_sentences(t, [t])[0]

I want rules s.t. the decile > -7.15. The current bottom 10% of a rule is always a lower bound on the decile, so if I see applies / 10 with score < -7.15 I can stop computing scores for that rule

In [34]:
# here
ps_scores = {}

In [35]:
# here
ps.last = None

In [36]:
# here
rule_scores = []
rejected = set()
print(len(really_frequent_rules))
for idx, i in enumerate(really_frequent_rules):
    orig_texts =  [right_val[z] for z in rule_applies[i]]
    orig_scor = [orig_scores[z] for z in rule_applies[i]]
    scores = np.ones(len(orig_texts)) * -50
#     if idx in rejected:
#         rule_scores.append(scores)
#         continue
    decile = np.ceil(.1 * len(orig_texts))
    new_texts = rule_other_texts[i]
    bad_scores = 0
    for j, (o, n, orig) in enumerate(zip(orig_texts, new_texts, orig_scor)):
        if o not in ps_scores:
            ps_scores[o] = {}
        if n not in ps_scores[o]:
            if n == '':
                score = -40
            else:
                score = ps.score_sentences(o, [n])[0]
            ps_scores[o][n] = min(0, score - orig)
        scores[j] = ps_scores[o][n]
        if ps_scores[o][n] < threshold:
            bad_scores += 1
        if bad_scores >= decile:
            rejected.add(idx)
            break
    rule_scores.append(scores)
            
    if i % 1000 == 0:
        print(i)

171442
0
1000
2000
3000
4000
6000
7000
10000
11000
12000
13000
14000
15000
17000
18000
20000
21000
22000
23000
24000
25000
26000
27000
28000
30000
31000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
51000
52000
53000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
67000
68000
69000
70000
72000
73000
74000
75000
76000
78000
79000
80000
81000
82000
84000
85000
89000
91000
92000
93000
94000
95000
96000
97000
98000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
119000
120000
121000
122000
123000
125000
126000
127000
128000
130000
131000
132000
133000
135000
136000
137000
138000
139000
140000
142000
143000
144000
145000
146000
147000
148000
149000
153000
154000
157000
158000
159000
160000
161000
162000
164000
165000
166000
167000
168000
169000
171000
172000
173000
174000
176000
177000
179000
180000
183000
187000
188000
189000
191000
195000
196000
197000
198000
199000
20000

In [37]:
# import pickle
# pickle.dump({'ps_scores': ps_scores, 'orig_scores': orig_scores}, open('/home/marcotcr/tmp/polarity_scoresz.pkl', 'wb'))

In [38]:
# len(rule_scores) - len(rejected)

In [39]:
# here
rule_flip_scores = [rule_scores[i][rule_other_flips[really_frequent_rules[i]]] for i in range(len(rule_scores))]

In [43]:
# here
frequent_flips = [np.array(rule_applies[i])[rule_other_flips[i]] for i in really_frequent_rules]

In [46]:
# here
rule_precsupports = [len(rule_applies[i]) for i in really_frequent_rules]

In [47]:
# here
from rule_picking import disqualify_rules
threshold=-7.15
# x = choose_rules_coverage(fake_scores, frequent_flips, frequent_supports,
disqualified = disqualify_rules(rule_scores, frequent_flips,
                          rule_precsupports, 
                      min_precision=0.0, min_flips=6, 
                         min_bad_score=threshold, max_bad_proportion=.10,
                          max_bad_sum=999999)

In [ ]:
# [(i, x.hash()) for (i, x) in enumerate(frequent_rules) if 'text_movie -> text_film' in x.hash()]

In [48]:
# here
# finalize SEARs
from rule_picking import choose_rules_coverage
threshold=-7.15
a = time.time()
x = choose_rules_coverage(rule_flip_scores, frequent_flips, None,
                          None, len(right_preds),
                                frequent_scores_on_all=None, k=10, metric='max',
                      min_precision=0.0, min_flips=0, exp=True,
                         min_bad_score=threshold, max_bad_proportion=.1,
                          max_bad_sum=999999,
                         disqualified=disqualified,
                         start_from=[])
print(time.time() -a)
support_denominator = float(len(right_preds))
soup = lambda x: len(rule_applies[really_frequent_rules[x]]) / support_denominator 
prec = lambda x: frequent_flips[x].shape[0] / float(len(rule_scores[x]))
fl = len(set([a for r in x for a in frequent_flips[r]]))
print('Instances flipped: %d (%.2f)' % (fl, fl / float(len(right_preds))))
print('\n'.join(['%-5d %-5d %-5d %-35s f:%d avg_s:%.2f bad_s:%.2f bad_sum:%d Prec:%.2f Supp:%.2f' % (
                i, x[i], really_frequent_rules[r],
                frequent_rules[really_frequent_rules[r]].hash().replace('text_', '').replace('pos_', '').replace('tag_', ''),
                frequent_flips[r].shape[0],
                np.exp(rule_flip_scores[r]).mean(), (rule_scores[r] < threshold).mean(),
                (rule_scores[r] < threshold).sum(), prec(r), soup(r)) for i, r in enumerate(x)]))


0.3288688659667969
Instances flipped: 77 (0.10)
0     6000  7245  film -> movie                       f:17 avg_s:0.57 bad_s:0.03 bad_sum:3 Prec:0.15 Supp:0.15
1     56940 69756 movie -> film                       f:10 avg_s:0.75 bad_s:0.06 bad_sum:7 Prec:0.08 Supp:0.16
2     3588  4482  acting -> performance               f:6 avg_s:0.70 bad_s:0.05 bad_sum:2 Prec:0.15 Supp:0.05
3     129143 175592 PRON is DET -> it 's DET            f:7 avg_s:0.60 bad_s:0.06 bad_sum:1 Prec:0.44 Supp:0.02
4     91380 119755 DET movie -> the film               f:7 avg_s:0.56 bad_s:0.07 bad_sum:6 Prec:0.08 Supp:0.12
5     6003  7248  this film -> this movie             f:6 avg_s:0.92 bad_s:0.03 bad_sum:1 Prec:0.21 Supp:0.04
6     27748 34267 great -> major                      f:7 avg_s:0.14 bad_s:0.07 bad_sum:2 Prec:0.26 Supp:0.04
7     26527 31957 is -> was                           f:32 avg_s:0.02 bad_s:0.04 bad_sum:9 Prec:0.13 Supp:0.31
8     92025 120528 It is -> He 's                      f:6 avg_s:0

### a couple of examples from the first rules

In [49]:
# here
for r in x:
    rid = really_frequent_rules[r]
    rule =  frequent_rules[rid]
    print('Rule: %s' % rule.hash())
    print()
    for f in rule_flips[rid][:2]:
        print('%s\nP(positive):%.2f' % (right_val[f], model.predict_proba([right_val[f]])[0, 1]))
        print()
        new = rule.apply(token_right[f])[1]
        print('%s\nP(positive):%.2f' % (new, model.predict_proba([new])[0, 1]))
        print()
        print()
    print('---------------')

Rule: text_film -> text_movie

Yes , this film does require a rather significant amount of puzzle - solving , but the pieces fit together to create a beautiful picture .
P(positive):0.58

Yes , this movie does require a rather significant amount of puzzle - solving , but the pieces fit together to create a beautiful picture .
P(positive):0.37


Overall , the film is interesting and thought - provoking .
P(positive):0.67

Overall , the movie is interesting and thought - provoking .
P(positive):0.50


---------------
Rule: text_movie -> text_film

A very , very , very slow - moving , aimless movie about a distressed , drifting young man .
P(positive):0.40

A very , very , very slow - moving , aimless film about a distressed , drifting young man .
P(positive):0.54


Yeah , the movie pretty much sucked .
P(positive):0.31

Yeah , the film pretty much sucked .
P(positive):0.51


---------------
Rule: text_acting -> text_performance

The acting is terrible , and the writing is worse .
P(posit

In [124]:
question = 'Who is watching a movie?'
token_q = tokenizer.tokenize([question])
new = rule.apply(token_q[0])
print(rule.hash())
print(new)

# token_q
# token_right[0]

pos_DET tag_NN -> text_the tag_NN
(True, 'Who is watching the movie ?')


In [129]:
def paraphrase_instance(instance, topk=10, threshold=-10, ):
#     orig_pred = model.predict([instance])[0]
    instance_for_onmt = onmt_model.clean_text(' '.join([x.text for x in nlp.tokenizer(instance)]), only_upper=False)
    paraphrases = ps.generate_paraphrases(instance_for_onmt, topk=topk, edit_distance_cutoff=4, threshold=threshold)
    texts = tokenizer.clean_for_model(tokenizer.clean_for_humans([x[0] for x in paraphrases]))
    return texts
#     preds = model.predict(texts)
#     fs = [(texts[i], paraphrases[i][1]) for i in np.where(preds != orig_pred)[0]]
#     return fs

paraphrase_instance('The plane was late and the detectives were waiting at the airport all morning.')

['The plane was late and the detectives were waiting for the airport every morning .',
 'The plane was late and the detectives were waiting for the airport all morning .',
 'The airplane was late and the detectives were waiting for the airport every morning .',
 'The plane was late and the detectives were waiting at the airport all morning .',
 'The plane was late and the detectives were waiting at the airport every morning .',
 'The airplane was late and the detectives were waiting for the airport all morning .',
 'The plane was late and detectives were waiting for the airport every morning .',
 'The plane was late and the detectives were expecting the airport every morning .',
 'The plane was late , and the detectives were waiting for the airport every morning .',
 'The plane was late , and the detectives were waiting for the airport all morning .']

In [134]:
with open('rule_scores.pkl', 'rb') as f:
    new_rule_scores = pickle.load(f)

In [142]:
all([(a==b).all() for a, b in zip(new_rule_scores, rule_scores)])

True